In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import statistics


import glob

import matplotlib.pyplot as plt

In [ ]:
#base_df = pd.read_csv("/data/schwurbelarchiv/extracted_information/dataframes/base_df.csv.gzip", compression = 'gzip')
#group_info = read_csv("/data/schwurbelarchiv/extracted_information/dataframes/group_info.csv.gzip", compression = 'gzip')

In [ ]:
summary_df = group_info.tail(1)

In [ ]:
group_info.drop(group_info.tail(1).index,inplace=True)

In [ ]:
max(group_info.n_authors)

In [ ]:
np.percentile(group_info.n_authors, 50)

In [ ]:
sum(group_info.n_authors)/len(group_info.n_authors)

In [ ]:
st.t.interval(alpha=0.90, df=len(group_info.n_authors)-1,
              loc=np.mean(group_info.n_authors),
              scale=st.sem(group_info.n_authors))

In [ ]:
group_info = pd.concat([group_info, summary_df], axis = 0)

In [ ]:
group_info = group_info.reset_index(drop=True)

In [ ]:
posting_frequency = base_df.groupby("day").count()
posting_frequency = posting_frequency[["posting_date"]]

In [ ]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe'

fig = px.line(posting_frequency, x=posting_frequency.index, y='posting_date')
fig.show()

In [ ]:
week_postings = base_df[["group_name", "posting_date", "weekday"]].copy()

In [ ]:
week_postings["hour"] = week_postings["posting_date"].dt.hour

In [ ]:
days_grouped = week_postings.groupby(["weekday", "hour"]).size()
days_grouped = days_grouped.unstack()

In [ ]:
plt.imshow(days_grouped, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
week_postings.pivot_table(index="hour", columns="weekday", aggfunc="size").plot(kind="line")
plt.show()

In [ ]:
hours_passing = []
for i in range(7):
    hours_passing += days_grouped.iloc[i].tolist()
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
hours_list = [4, 8, 12, 16, 20]
the_week = []
for day in weekdays:
    the_week.append(day)
    the_week += hours_list

weekly_posting_df = pd.DataFrame({"posting_numbers":hours_passing})

In [ ]:
fig = px.line(weekly_posting_df, x=weekly_posting_df.index, y="posting_numbers")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 168, 4)),
        ticktext = the_week
    )
)
fig.show()

In [ ]:
def link_removal(some_string):
    some_string = re.sub("<a.*?a>", "", some_string)
    some_string = re.sub("<code>.*?</code>", "", some_string)
    some_string = re.sub("<.*?>", "", some_string)

    return some_string

In [ ]:
base_df['day'] = base_df['posting_date'].dt.floor('D') + pd.Timedelta(12, unit = 'h')

In [ ]:
#analysis of peak on January 7th
base_df[base_df['day'] == '2021-01-07 12:00:00+00:00']['group_name'].value_counts().head(10)

In [ ]:
#base_df[(base_df['group_name'] == 'O-Ton TV - Der Infokanal (SHAEF/2Q2Q/WWG1WGA/DEFENDER/TRUMP/2Q21/GESARA)') & (base_df['day'] == '2021-06-12 12:00:00+00:00')]['message'].dropna().head(50)#['message'].value_counts

In [ ]:
group_names = base_df["group_name"].unique().tolist()
individual_dates = list(base_df['day'].unique())

len(group_names)
len(individual_dates)


text_df = pd.DataFrame(columns=["group_name", "day", "messages"]) # Add if working with whisper transcriptions , "transcribed_message"])
count = 0
for n in range(len(group_names)):
    group = group_names[n]
    for i in range(len(individual_dates)):
        build_text = base_df.loc[(base_df["group_name"] == group_names[n]) & (base_df["day"] == individual_dates[i])]
        count += 1
        if len(build_text) > 0:
            day = individual_dates[i]
            build_text["message"] = build_text["message"].fillna("")
            build_text["message"] = build_text["message"].apply(link_removal)
            build_text["fwd_message"] = build_text["fwd_message"].fillna("")
            build_text["fwd_message"] = build_text["fwd_message"].apply(link_removal)
            #build_text["transcribed_message"] = build_text["transcribed_message"].fillna("")
            #build_text["transcribed_message"] = build_text["transcribed_message"].apply(link_removal)
            messages = " ".join(build_text["message"].to_list())+" " + " ".join(build_text["fwd_message"].to_list()) # + " ".join(build_text["transcribed_message"].to_list())

            new_row = len(text_df)
            text_df.loc[new_row] = [group, day, messages]
        print(count)




In [ ]:
import datetime;

groupy = base_df.groupby(by=["group_name", "day"])

ct = datetime.datetime.now()
print("current time:-", ct)

all_mess = groupy["message", "fwd_message"].agg(lambda x: ' '.join(x.dropna()))
text_df = all_mess.reset_index()

ct = datetime.datetime.now()
print("current time:-", ct)

text_df["messages"] = text_df["message"] + text_df["fwd_message"]
ct = datetime.datetime.now()
print("current time:-", ct)
text_df["messages"] = text_df["messages"].fillna("")
ct = datetime.datetime.now()
print("current time:-", ct)
text_df["messages"] = text_df["messages"].apply(link_removal)
ct = datetime.datetime.now()
print("current time:-", ct)
text_df = text_df[text_df["messages"] != ""]
ct = datetime.datetime.now()
print("current time:-", ct)
text_df = text_df[~(text_df["messages"].str.strip() =="")].reset_index()
text_df

In [ ]:
#text_df.to_csv("/data/schwurbelarchiv/extracted_information/dataframes/text_df.csv.gzip", compression = 'gzip')